<a href="https://colab.research.google.com/github/arkeodev/time-series/blob/main/Statistical_Time_Series_Analysis/05-llr-test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 05-LLR Test

The LLR test is used in statistics to compare the fit of two models, typically a simpler (nested) model against a more complex one. Here is the code:

### Implementation

In [3]:
import numpy as np
from scipy.stats import chi2
import statsmodels.api as sm

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

def LLR_test(mod_1, mod_2, DF=1):
    """
    Perform the Likelihood Ratio Test (LLR) to compare two statistical models.
    
    Parameters
    ----------
    mod_1 : statsmodels object
        A statistical model (e.g., ARIMA) that implements .fit().llf.
    mod_2 : statsmodels object
        A second statistical model (e.g., ARIMA) that implements .fit().llf.
    DF : int, optional
        Degrees of freedom (difference in parameters between the two models).
        Default is 1.

    Returns
    -------
    p : float
        The p-value obtained from the chi-squared distribution.
    """
    # Fit both models and extract the log-likelihoods
    L1 = mod_1.fit().llf  # log-likelihood of the first (simpler) model
    L2 = mod_2.fit().llf  # log-likelihood of the second (more complex) model
    
    # Compute the Likelihood Ratio statistic
    # LR = 2 * (log-likelihood of complex model - log-likelihood of simple model)
    LR = 2 * (L2 - L1)
    
    # Compute the p-value using the survival function of the chi-squared distribution
    # with DF degrees of freedom
    p = chi2.sf(LR, DF).round(3)
    return p

# -------------------- Example Usage --------------------
if __name__ == "__main__":
    # For demonstration, we'll compare two ARIMA models on some macroeconomic data
    # from statsmodels. The dataset includes real GDP over time.
    
    # Load the dataset
    data = sm.datasets.macrodata.load_pandas().data['realgdp']
    
    # Create two ARIMA models with different orders
    # Note: ARIMA(order=(p, d, q)) => p=AR terms, d=differences, q=MA terms
    mod_1 = sm.tsa.ARIMA(data, order=(1, 0, 0))  # AR(1)
    mod_2 = sm.tsa.ARIMA(data, order=(2, 0, 0))  # AR(2)
    
    # For these models, the difference in number of parameters is 1
    # (because AR(2) has one extra AR parameter compared to AR(1))
    p_value = LLR_test(mod_1, mod_2, DF=1)
    
    # Print the p-value to see if the more complex model (AR(2)) is significantly better
    print(f"LLR Test p-value between AR(1) and AR(2): {p_value}")

LLR Test p-value between AR(1) and AR(2): 0.0


The result is a p-value that tells you whether the additional complexity of `complex` significantly improves the fit of the model compared to `simple`. If the p-value is less than a chosen significance level (usually 0.05), you would reject the null hypothesis that `simple` is true, indicating that `complex` provides a significantly better fit to the data.

### Degrees of Freedom

The degrees of freedom (DF) in the context of the Likelihood Ratio Test (LLR test) is the difference in the number of estimated parameters between the two models being compared.

For ARIMA models, the parameters are:

- The number of AR (autoregressive) terms (p)
- The number of differences (d)
- The number of MA (moving average) terms (q)

In an ARIMA(p, d, q) model, the total number of parameters is `p + q + 1` if a constant or trend term is included in the model, or `p + q` if it is not.

Let's think that complex model is the `ARIMA(4,1,2)` model, if a constant is included, there would be `4 (AR) + 2 (MA) + 1 (constant) = 7` parameters.

For the simple model `ARIMA(1,1,1)` model, again assuming a constant is included, there would be `1 (AR) + 1 (MA) + 1 (constant) = 3` parameters.

The degrees of freedom for the LLR test comparing these two models would then be the difference in the number of parameters:

DF = Parameters of ARIMA(4,1,2) - Parameters of ARIMA(1,1,1)
DF = 7 - 3
DF = 4

Therefore, it will be uses `DF = 4` for the LLR test between `ARIMA(4,1,2)` and `ARIMA(1,1,1)`. This represents the additional parameters that `ARIMA(4,1,2)` is using over `ARIMA(1,1,1)`.

### Nested vs Non-nested Models

For the Likelihood Ratio Test (LLR test) to be valid, the models being compared should ideally be nested. This means that one model is a special case of the other when certain parameters are set to zero.

Nested models typically have the following relationship:

- They are estimated on the same dataset.
- The more complex model (with more parameters) reduces to the simpler model under certain parameter restrictions (usually setting some parameters to zero).

Now, let's consider the `ARIMA(5,1,1)` and `ARIMA(2,1,4)` models:

- `ARIMA(5,1,1)` has 5 AR terms, 1 differencing term, and 1 MA term.
- `ARIMA(2,1,4)` has 2 AR terms, 1 differencing term, and 4 MA terms.

The number of parameters for each (assuming both have a constant term) is:

- For `ARIMA(5,1,1)`: 5 (AR) + 1 (MA) + 1 (constant) = 7 parameters
- For `ARIMA(2,1,4)`: 2 (AR) + 4 (MA) + 1 (constant) = 7 parameters

Since both models have the same number of parameters, they have the same degrees of freedom when it comes to comparing the estimated parameters. However, they are not nested models because `ARIMA(5,1,1)` cannot be reduced to `ARIMA(2,1,4)` or vice versa by just setting some parameters to zero. Each has different lag orders for the AR and MA components.

Therefore, a traditional LLR test may not be appropriate for comparing `ARIMA(5,1,1)` and `ARIMA(2,1,4)` because they are not nested. 

However, if you still wanted to compare models using some form of likelihood-based approach, you might look at other criteria like Akaike’s Information Criterion (AIC) or Bayesian Information Criterion (BIC), which are suitable for comparing non-nested models. They penalize the likelihood for the number of parameters, so the model with the lower AIC or BIC is considered better in terms of the trade-off between goodness-of-fit and complexity.